In [1]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, ReLU, MaxPooling2D, Lambda, LSTM, Input, Layer
from keras.layers import concatenate, Dense, Activation, Dropout, Reshape, Flatten
from Spatial_Model import create_model
from keras import optimizers
from keras import losses
from keras import metrics
from keras import backend as K
import os
import numpy as np

model = create_model()
model.summary()

Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 21, 31, 5)    0                                            
__________________________________________________________________________________________________
conv1_h1 (Conv2D)               (None, 21, 31, 64)   8064        input_3[0][0]                    
__________________________________________________________________________________________________
conv1_h2 (Conv2D)               (None, 21, 31, 64)   8064        input_3[0][0]                    
__________________________________________________________________________________________________
conv1_h3 (Conv2D)               (None, 21, 31, 64)   8064        input_3[0][0]                    
__________________________________________________________________________________________________
MaxPool_h1

In [ ]:
import os
import numpy as np
from datetime import datetime , timedelta
from random import shuffle

def load_metadata(grid_data_path,air_data_path):
    metadata = []
    grid_f = os.listdir(grid_data_path)
    air_f = os.listdir(air_data_path)
    for f in air_f:
        #Check file extension. Allow only 'npy' files.
        ext = os.path.splitext(f)[1]
        if ext == '.npy':
            x_open = f.find('[')
            x_close = f.find(']')
            date_curr_str = f[x_open+1:x_close]
            date_curr = datetime.strptime(date_curr_str, "%Y-%m-%d %H.%M.%S")
            date_prev_1 = date_curr - timedelta(hours = 1)
            date_prev_2 = date_curr - timedelta(hours = 2)
            date_prev_1_str = date_prev_1.strftime("%Y-%m-%d %H.%M.%S")
            date_prev_2_str = date_prev_2.strftime("%Y-%m-%d %H.%M.%S")
            if 'grid_map' + f[x_open:x_close+1] + '.npy' in grid_f:
                if 'air_map[' + date_prev_1_str + '].npy' in air_f:
                    if 'air_map[' + date_prev_2_str + '].npy' in air_f:
                        metadata.append([f[x_open+1:x_close],
                                         os.path.join(air_data_path,'air_map[' + date_prev_2_str + '].npy'),
                                         os.path.join(air_data_path,'air_map[' + date_prev_1_str + '].npy'),
                                         os.path.join(grid_data_path,'grid_map[' +  date_prev_1_str + '].npy'),
                                         os.path.join(air_data_path,f)])
    return metadata

grid_data = "C:/Users/Juliana/Desktop/MSBD5002 Project/grid_data/"
air_data = "C:/Users/Juliana/Desktop/MSBD5002 Project/air_data/"
data = load_metadata(grid_data,air_data)

def generator(data, min_index, max_index, shuffle=False, batch_size=128):
    if max_index is None:
        max_index = len(data) - 1
    
    i = min_index
    
    while 1:
        if shuffle:
            rows = np.random.randint(min_index, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)
        
        grid_samples = np.zeros((len(rows),21,31,5))
        air_targets = np.zeros((len(rows),3,35))
        air_samples_prev_1 = np.zeros((len(rows),3,35))
        air_samples_prev_2 = np.zeros((len(rows),3,35))
        
        for j, row in enumerate(rows):
            air_samples_prev_2[j] = np.load(data[rows[j]-1][1])
            air_samples_prev_1[j] = np.load(data[rows[j]-1][2])
            grid_samples[j] = np.load(data[rows[j]-1][3])
            air_targets[j] = np.load(data[rows[j]-1][4])
        
        yield [air_samples_prev_2, air_samples_prev_1, grid_samples], air_targets
        
#model.load_weights('spatial_weight.h5')

batch_size = 50
[min_index_train, max_index_train] = [0, len(data)-1]
[min_index_val, max_index_val] = [int(len(data)/2)+1, len(data)-1]

train_gen = generator(data, min_index = min_index_train, max_index = max_index_train, shuffle=False, batch_size=batch_size)
val_gen = generator(data, min_index = min_index_val, max_index = max_index_val, shuffle = False, batch_size=batch_size)

model.compile(optimizer=optimizers.Adam(lr=0.002, beta_1=0.9, beta_2=0.999), loss=losses.MAE)
model.fit_generator(train_gen, steps_per_epoch=50, epochs=50, shuffle = False, validation_data = val_gen, validation_steps=50, verbose=1)

In [39]:
model.save_weights('spatial_model.h5')